# Modeling and Simulation in Python

Chapter 25

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [2]:
def plot_theta(results):
    plot(results.theta, color='C0', label='theta')
    decorate(xlabel='Time (s)',
             ylabel='Angle (rad)')
    


In [3]:
def plot_y(results):
    plot(results.y, color='C1', label='y')

    decorate(xlabel='Time (s)',
             ylabel='Length (m)')
    


In [4]:
def plot_r(results):
    plot(results.r, color='C2', label='r')

    decorate(xlabel='Time (s)',
             ylabel='Radius (mm)')
    


### Unrolling

We'll start by loading the units we need.

In [5]:
radian = UNITS.radian
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
N = UNITS.newton

And a few more parameters in the `Params` object.

In [6]:
params = Params(Rmin = 0.02 * m,
                Rmax = 0.055 * m,
                Mcore = 15e-3 * kg,
                Mroll = 215e-3 * kg,
                L = 47 * m,
                tension = 2e-4 * N,
                t_end = 180 * s)

`make_system` computes `rho_h`, which we'll need to compute moment of inertia, and `k`, which we'll use to compute `r`.

In [7]:
def make_system(params):
    """Make a system object.
    
    params: Params with Rmin, Rmax, Mcore, Mroll,
                              L, tension, and t_end
    
    returns: System with init, k, rho_h, Rmin, Rmax,
                         Mcore, Mroll, ts
    """
    unpack(params)
    
    init = State(theta = 0 * radian,
                 omega = 0 * radian/s,
                 y = L)
    
    area = pi * (Rmax**2 - Rmin**2)
    rho_h = Mroll / area
    k = (Rmax**2 - Rmin**2) / 2 / L / radian    
    
    return System(init=init, k=k, rho_h=rho_h,
                  Rmin=Rmin, Rmax=Rmax,
                  Mcore=Mcore, Mroll=Mroll, 
                  t_end=t_end)

Testing `make_system`

In [8]:
system = make_system(params)

In [9]:
system.init

Here's how we compute `I` as a function of `r`:

In [10]:
def moment_of_inertia(r, system):
    """Moment of inertia for a roll of toilet paper.
    
    r: current radius of roll in meters
    system: System object with Mcore, rho, Rmin, Rmax
    
    returns: moment of inertia in kg m**2
    """
    unpack(system)
    Icore = Mcore * Rmin**2   
    Iroll = pi * rho_h / 2 * (r**4 - Rmin**4)
    return Icore + Iroll

When `r` is `Rmin`, `I` is small.

In [11]:
moment_of_inertia(system.Rmin, system)

As `r` increases, so does `I`.

In [12]:
moment_of_inertia(system.Rmax, system)

Here's the slope function.

In [13]:
def slope_func(state, t, system):
    """Computes the derivatives of the state variables.
    
    state: State object with theta, omega, y
    t: time
    system: System object with Rmin, k, Mcore, rho_h, tension
    
    returns: sequence of derivatives
    """
    theta, omega, y = state
    unpack(system)
    
    r = sqrt(2*k*y + Rmin**2)
    I = moment_of_inertia(r, system)
    tau = r * tension
    alpha = tau / I
    dydt = -r * omega
    
    return omega, alpha, dydt      

Testing `slope_func`

In [14]:
slope_func(system.init, 0*s, system)

Now we can run the simulation.

In [15]:
results, details = run_ode_solver(system, slope_func)
details

And look at the results.

In [16]:
results.tail()

Plotting `theta`

In [17]:
plot_theta(results)

Plotting `omega`

In [18]:
def plot_omega(results):
    plot(results.omega, color='orange', label='omega')

    decorate(xlabel='Time (s)',
             ylabel='Angular velocity (rad/s)')
    
plot_omega(results)

Plotting `y`

In [19]:
plot_y(results)

Here's the figure from the book.

In [20]:
subplot(3, 1, 1)
plot_theta(results)

subplot(3, 1, 2)
plot_omega(results)

subplot(3, 1, 3)
plot_y(results)

savefig('figs/chap11-fig02.pdf')

### Yo-yo

**Exercise:**  Simulate the descent of a yo-yo.  How long does it take to reach the end of the string?

I provide a `Params` object with the system parameters:

* `Rmin` is the radius of the axle.  `Rmax` is the radius of the axle plus rolled string.

* `Rout` is the radius of the yo-yo body.  `mass` is the total mass of the yo-yo, ignoring the string.  

* `L` is the length of the string.

* `g` is the acceleration of gravity.

In [21]:
params = Params(Rmin = 8e-3 * m,
                Rmax = 16e-3 * m,
                Rout = 35e-3 * m,
                mass = 50e-3 * kg,
                L = 1 * m,
                g = 9.8 * m / s**2,
                t_end = 1 * s)

Here's a `make_system` function that computes `I` and `k` based on the system parameters.

I estimated `I` by modeling the yo-yo as a solid cylinder with uniform density ([see here](https://en.wikipedia.org/wiki/List_of_moments_of_inertia)).  In reality, the distribution of weight in a yo-yo is often designed to achieve desired effects.  But we'll keep it simple.

In [22]:
def make_system(params):
    """Make a system object.
    
    params: Params with Rmin, Rmax, Rout, 
                              mass, L, g, t_end
    
    returns: System with init, k, Rmin, Rmax, mass,
                         I, g, ts
    """
    unpack(params)
    
    init = State(theta = 0 * radian,
                 omega = 0 * radian/s,
                 y = L,
                 v = 0 * m / s)
    
    I = mass * Rout**2 / 2
    k = (Rmax**2 - Rmin**2) / 2 / L / radian    
    
    return System(init=init, k=k,
                  Rmin=Rmin, Rmax=Rmax,
                  mass=mass, I=I, g=g,
                  t_end=t_end)

Testing `make_system`

In [23]:
system = make_system(params)

In [24]:
system.init

Write a slope function for this system, using these results from the book:

$ r = \sqrt{2 k y + R_{min}^2} $ 

$ T      = m g I / I^*  $

$ a      = -m g r^2 / I^* $

$ \alpha  = m g r / I^*  $

where $I^*$ is the augmented moment of inertia, $I + m r^2$.

Hint: If `y` is less than 0, it means you have reached the end of the string, so the equation for `r` is no longer valid.  In this case, the simplest thing to do it return the sequence of derivatives `0, 0, 0, 0`

In [25]:
# Solution goes here

Test your slope function with the initial paramss.

In [26]:
slope_func(system.init, 0*s, system)

Write an event function that will stop the simulation when `y` is 0.

In [27]:
# Solution goes here

Test your event function:

In [28]:
event_func(system.init, 0*s, system)

Then run the simulation.

In [29]:
results, details = run_ode_solver(system, slope_func, events=event_func)
details

Check the final state.  If things have gone according to plan, the final value of `y` should be close to 0.

In [30]:
results.tail()

Plot the results.

In [31]:
t_final = get_last_label(results)
ts = linspace(0, t_final, 101)
results, details = run_ode_solver(system, slope_func, t_eval=ts)
details

`theta` should increase and accelerate.

In [32]:
plot_theta(results)

`y` should decrease and accelerate down.

In [33]:
plot_y(results)